In [49]:
import json
import pandas as pd
import re
import requests


#JSON File für die Ladestationen als speichern
url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/oicp/ch.bfe.ladestellen-elektromobilitaet.json"

#Daten mittels REST GET request holen
data = requests.get(url)

#Daten als JSON File speichern
data_orig = data.json()

# Read the data
#data_orig = json.load(open('ch.bfe.ladestellen-elektromobilitaet.json', 
#                           encoding='utf-8'))

# Save the data as pandas data frame
data = pd.DataFrame(data_orig["EVSEData"])

# Getting the index of evpass
idx = data[data["OperatorName"].isin(['evpass'])].index[0]

# Create data frame from column 'EVSEDataRecord' in the subset
df = pd.DataFrame(data.loc[idx, 'EVSEDataRecord'])


# Look at the geo-coordinates (latitude and longitude)
coordinates = df.GeoCoordinates

# Getting single values from as string
coord_str = "".join(coordinates[0].values())

# Splitting string by white space
re.findall(r'\S+', coord_str)

# Now loop though the coordinates object
out_list = []
for i in coordinates:
    # Getting single values from as string
    coord_str = "".join(i.values())
    # Splitting string by white space
    coords = re.findall(r'\S+', coord_str)
    out_list.append(coords)
    

# Create data frame from the list
df = pd.DataFrame(out_list, columns =['Latitude', 'Longitude'])

# Here we have still strings of the which need to be converted to integers

# First: exclude white space using pythons lambda function
df['Latitude'] = df['Latitude'].apply(lambda x: x.strip())
df['Longitude'] = df['Longitude'].apply(lambda x: x.strip())

# Second: cange to integer values
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] =  df['Longitude'].astype(float)





In [52]:
import folium

m = folium.Map(location=[46.4754, 8.1355], zoom_start = 7)

#Loop trough datasets and set a marker for every charging station from the bfe dataset
for i in range(len(df)):
    x = df.Latitude[i]
    y = df.Longitude[i]
    folium.Marker(
    [x, y]).add_to(m)

m
    

